# REGRESSION FOR ALGORITHMIC MUSIC COMPOSITION

## Extracting chords from ChordPro files of The Beatles Songs.

In this chapter, the functions used for parsing ChordPro files were defined. An organized library version of the functions also can be found in the repository: https://github.com/aysimakarca/chordpro. 

Data used was collected from http://getsome.org/guitar/olga/chordpro/b/Beatles/

The required  python libraries for the notebook to run are listed below:
- pandas
- numpy
- mathplotlib
- sklearn
- pychord
- prettiy_midi
- flask

In [1]:
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from pychord import Chord

Here we define the unique note names as the limited characters which an chord can start with.

In [2]:
uniqueChordNames = ["A","B", "C", "D", "E", "F", "G"]       


The functions below are part of ChordPro parsing features. Here we extract special information like chorus, intro, title, key and using them in the creation of Song object below. 

In [3]:
def readChordProFile(chordProFile):
    with open(chordProFile, 'r') as file:
        print(chordProFile)
        content = file.read()
        return content

In [4]:
def cleanChords(chords):
    finalChords = []
    for chord in chords:
        chord = chord.replace("H", "B")
        chord = chord.replace("b", "-")
        chord = chord.replace("min", "m")
        chord = chord.replace("maj", "")
        chord = chord.replace("sus", "")
        chord = chord.replace("dim", "")
        if len(chord) > 3:
            chord = chord[0:3]
        chord = chord.replace("7", "")        
        chord = chord.replace("(", "")
        chord = chord.replace("+", "")
        chord = chord.replace("?", "")
        chord = chord.replace(".", "")
        chord = chord.split("/")[0]
        chord = chord.split("+")[0]
        chord = re.sub(r'\d+', '', chord)
        restWithoutAlphabethic = re.sub(r'[a-zA-Z]+', '', chord[1:]) 
        chord = chord[0] + restWithoutAlphabethic
        
        if chord[0:1] in uniqueChordNames:
            finalChords.append(chord)
    return finalChords

In [5]:
def getChordsOfALine(line, content):
    chordPattern = r'\[([^\]]+)\]'
    lineChords = []
    if "chorus}" in line.lower():
        lineChords = getChorus(content)
    elif "repeat intro}" in line.lower():
        lineChords = getIntro(content)
    elif "repeat bridge}" in line.lower():
        lineChords = getBridge(content)
    elif "[" in line:
        lineChords = re.findall(chordPattern, line)   
    return lineChords

In [6]:
def getChorus(content):
    patternArr = ["{soc}(.*?){eoc}", "{start_of_chorus}(.*?){end_of_chorus}"]
    chords = []
    for pattern in patternArr:
        match = re.search(pattern, content, re.DOTALL)
        if match:
            chorus = match.group(1).strip()
            chordPattern = r"\[([^\]]*)\]"
            chords = re.findall(chordPattern, chorus)
            return chords
    return chords

In [7]:
def getIntro(content):
    chordsOfIntro = []
    inIntro = False
    for line in content.split("\n"): 
        if 'c:intro:}' in line.lower() or 'c:intro}'in line.lower():
            inIntro = True
        elif inIntro:
            lineChords = getChordsOfALine(line, content)
            if len(lineChords) > 0:
                chordsOfIntro.extend(lineChords)
            else:
                inIntro = False
                break
    return chordsOfIntro

In [8]:
def getBridge(content):
    chordsOfBridge = []
    inBridge = False
    for line in content.split("\n"): 
        if 'c:bridge:}' in line.lower() or '{c:bridge}'in line.lower():
            inBridge = True
        elif inBridge:
            lineChords = getChordsOfALine(line, content)
            if len(lineChords) > 0:
                chordsOfBridge.extend(lineChords)
            else:
                inBridge = False
                break
    return chordsOfBridge

In [9]:
def getTitle(content, chordProFile):
    pattern = r"\{title:\s*(.*?)\s*\}"
    match = re.search(pattern, content)
    title = ''
    if match:
        title = match.group(1)
    else:
        title = os.path.splitext(os.path.basename(chordProFile))[0]
    return title

In [10]:
def getKey(content):
    pattern = r"{key:\s*([A-G][#b]?)\s*}"
    match = re.search(pattern, content)
    key = ''
    if match:
        key = match.group(1)
    return key

A song class was created in order to access the chord information easily. A song object can be initialized by providing a path for the ChordPro song file and the constructor will call the functions above. It will extract the chords, chorus and other information within the song object.

In [11]:
class Song:
    def __init__(self, chordProFile):
        self.chordProFile = chordProFile
        self.content = readChordProFile(chordProFile)
        self.chorus = self.getChorus()
        self.chords = self.getChords()
        self.title = self.getTitle()
        self.key = self.getKey()
        
    def getChords(self):
        chords = []
        for line in self.content.split("\n"): 
            lineChords = getChordsOfALine(line, self.content)
            chords.extend(cleanChords(lineChords))
        return chords
    
    def getChordsBeforeChorus(self):
        chords = []
        for line in self.content.split("\n"): 
            if 'chorus' in line.lower() or '{soc}'in line.lower() or '{start_of_chorus}'in line.lower():
                break
            lineChords = getChordsOfALine(line, self.content)
            chords.extend(cleanChords(lineChords))
        return chords
        
    def getChorus(self):
        return getChorus(self.content)
    
    def getIntro(self):
        return getIntro(self.content)
    
    def getTitle(self):
        return getTitle(self.content, self.chordProFile)
    
    def getKey(self):
        return getKey(self.content)
    

## Feature extraction & data preparation for training

In this part, we are going to create feature and target arrays by using the ChordPro functions above as well as the helper functions below.

### Create a song dictionary of songs with chords

First, we start with providing a folder path where songs can be found and we are going to iterate over each song. For every file we find, we are going to initialize a Song object and add its chords into a dictionary named **songDict** where key is the song name and value is the chord array. 

In [12]:
def createChordDictFromChordProFiles(dataFolderPath):
    fileNames = os.listdir(dataFolderPath)
    chordsDict = {}
    longestChordLength = 0
    for file in fileNames:
        if '.DS_Store' in file:
            continue
        chordproFile = dataFolderPath + file
        song = Song(chordproFile)
        chordsDict[song.title] = song.chords
    return chordsDict

In [13]:
pathOfChordTxtFiles = './data/chordpro/'

songDict = createChordDictFromChordProFiles(pathOfChordTxtFiles)

./data/chordpro/HelloGoodbye.chopro
./data/chordpro/TheContinuingStoryOfBungalowBill.chopro
./data/chordpro/YellowSubmarine.chopro
./data/chordpro/StrawberryFieldsForever.2.chopro
./data/chordpro/GetBack.chopro
./data/chordpro/Blackbird.chopro
./data/chordpro/Help.chopro
./data/chordpro/ADayInALife.chopro
./data/chordpro/DontLetMeDown.chopro
./data/chordpro/FromMeToYou.chopro
./data/chordpro/SoldierOfLove.chopro
./data/chordpro/EleanorRigby.chopro
./data/chordpro/ChristmastimeIsHereAgain.chopro
./data/chordpro/WeCanWorkItOut.chopro
./data/chordpro/WithALittleHelpFromMyFriends.chopro
./data/chordpro/AllTogetherNow.chopro
./data/chordpro/LetItBe.2.chopro
./data/chordpro/TellMeWhy.chopro
./data/chordpro/Anna.chopro
./data/chordpro/AcrossTheUniverse.chopro
./data/chordpro/IAmAWalrus.chopro
./data/chordpro/WithALittleHelpFromMyFriends.2.chopro
./data/chordpro/IShouldHaveKnownBetter.chopro
./data/chordpro/ICallYourName.chopro
./data/chordpro/CantBuyMeLove.2.chopro
./data/chordpro/SheLovesYou

### Map the string chord names to integer values

The chords we worked until this point are in string format. However it is better for the regression to use integer values compared to string values. So we will create two mapping functions to map chords from string to int and from int to string.

In order to do the mapping, we build a list of unique chords exists in our songs so that we can use the index as the integer value. This will also be used when mapping from int to chord names back.

In [14]:
distinctChordNames = list(sorted({ele for val in songDict.values() for ele in val}))
print(distinctChordNames)
print(len(distinctChordNames))

['A', 'A-', 'B', 'B-', 'C', 'C#', 'D', 'D#', 'E', 'E-', 'F', 'F#', 'G', 'G#']
14


In [15]:
def mapChordNamesToIndexes(chords):
    indexArr = []
    for i, chord in enumerate(chords):
        indexArr.append(distinctChordNames.index(chord))
    return indexArr

In [16]:
def mapIndexesToChordNames(indexes):
    chords = []
    for i, index in enumerate(indexes):
        chords.append(distinctChordNames[index])
    return chords

Here we ignore the songs which has less than 10 chords as they are most likely to have a wrong formatting.

In [17]:
finalChordDict = {}
for song in songDict:
        chordArray = songDict[song]
        if len(chordArray) < 10:
            print('Chords are too short, ignoring the song ' + song)
        else:   
            finalChordDict[song] = mapChordNamesToIndexes(chordArray)
        
print(finalChordDict)

Chords are too short, ignoring the song ChristmastimeIsHereAgain
Chords are too short, ignoring the song Anna
{'Hello, Goodbye': [4, 12, 6, 8, 6, 8, 6, 12, 6, 4, 12, 4, 9, 12, 4, 10, 12, 4, 12, 6, 8, 6, 8, 6, 12, 6, 4, 12, 4, 9, 12, 4, 10, 12, 4, 12, 6, 8, 6, 8, 6, 12, 6, 4, 12, 4, 9, 12, 4, 10, 12, 4, 12, 6, 8, 6, 8, 6, 12, 6, 4, 12, 4, 9, 12, 4, 10, 12, 4, 9, 12], 'The Continuing Story of Bungalow Bill': [4, 12, 4, 10, 4, 10, 12, 0, 8, 0, 6, 0, 6, 8, 0, 4, 10, 12, 0, 4, 10, 8, 12, 0, 10, 4, 12, 4, 10, 4, 10, 12, 0, 8, 0, 6, 0, 6, 8, 0, 4, 10, 12, 0, 4, 10, 12, 8, 12, 0, 10, 4, 12, 4, 10, 4, 10, 12, 0, 8, 0, 6, 0, 6, 8, 0, 4, 10, 12, 0, 4, 10, 12, 8, 12, 0, 10], 'Yellow Submarine': [12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 12, 6, 12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 4, 12, 12, 6, 12, 6, 12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 4, 12, 8, 0, 4, 6, 12, 6, 12, 6, 12], 'Strawberry Fields Forever': [12, 6, 9, 4, 6, 8, 0, 6, 12, 6, 6,

### Create feature and target arrays for training

Here we create functions for two strategy 

First one is using a fixed length for each song and using this array of a song in the feature array. This is the function **getFeaturesAndTargetsOfFlatSong**

Second approach is to use window length and hop length in order to slice the chords of the songs and use each window as the feature arrays. By default, second option is selected. This is the function **getFeaturesAndTargetsByWindowAndHop**

In [18]:
def getFeatureAndTargetArrForEachWindow(chords, windowLength, hopSize, index):
    featureArray = chords[index: index + windowLength]
    targetArray = chords[index + windowLength: index + windowLength + 1]
    return featureArray, targetArray

In [19]:
def getFeaturesAndTargetsByWindowAndHop(chordsDictionary, windowSize, hopSize):
    features = np.empty((0, windowSize), dtype=np.ndarray)
    targets = np.empty((0, 1), dtype=np.ndarray)
    for song in chordsDictionary:
        chordArray = chordsDictionary[song]
        if len(chordArray) < windowSize + 1:
            print('Chords are shorter than number of targets so skipping the song: ' + song)
            continue
        indexProcessed = 0
        while len(chordArray) - indexProcessed > windowSize + 1:
            featureArr, targetArray = getFeatureAndTargetArrForEachWindow(chordArray, windowSize, hopSize, indexProcessed)
            featureArr = np.array(featureArr).reshape(1, len(featureArr))
            features = np.append(features, featureArr, axis=0)
            targetArray = np.array(targetArray).reshape(1, len(targetArray))
            targets = np.append(targets, targetArray, axis=0)
            indexProcessed = indexProcessed + hopSize

    features = np.array(features)
    targets = np.array(targets)
    print(features.shape)
    print(targets.shape)
    print('Slicing the song chords is done!')
    return features, targets

In [20]:
def fillChordArraysUntilDesiredLength(chordArr, desiredLength):
    finalArr = np.zeros(desiredLength, dtype=int)
    chordLength = len(chordArr)
    if chordLength < desiredLength:
        print('Chords are shorter than number of targets so repeating the chords: ' + song)
        quotient, remainder = divmod(desiredLength, chordLength) 
        print(str(chordLength) + " quotient " + str(quotient) + " remainder " + str(remainder))
        for step in range(quotient):
            finalArr[step*chordLength:(step+1)*chordLength] = chordArr
        finalArr[chordLength*quotient: desiredLength] = chordArr[0:remainder]
        print(len(finalArr))
    elif chordLength > desiredLength:
        finalArr = chordArr[0: desiredLength]
    else:
        finalArr = chordArr
    print(len(finalArr))
    print(finalArr)
    return finalArr

In [21]:
def getFeaturesAndTargetsOfFlatSong(chordsDictionary, desiredFeatureLength):
    features = np.zeros((len(chordsDictionary),desiredFeatureLength), dtype=int)
    targets = np.zeros((len(chordsDictionary),1), dtype=int)
    for i, song in enumerate(chordsDictionary):
        chordArray = chordsDictionary[song] 
        finalChords = fillChordArraysUntilDesiredLength(chordArray, desiredFeatureLength+1)
        features[i:] = finalChords[0:desiredFeatureLength]
        targets[i:] = finalChords[desiredFeatureLength:desiredFeatureLength+1]
    print(features.shape)
    print(targets.shape)
    print('Slicing the song chords is done!')
    return features, targets

Here we define which approach we want to train our model with. Alternatively, you can comment out getFeaturesAndTargetsByWindowAndHop function and use getFeaturesAndTargetsOfFlatSong. 

**featureLength** variable decides the fixed length of the arrays and used by getFeaturesAndTargetsOfFlatSong function.

**windowSize** and **hopSize** variables are used by getFeaturesAndTargetsByWindowAndHop functiona and decides the window and hop sizes while slicing the chords.

In [22]:
windowSize = 16
hopSize = 2

featureLength = 50

featureArr, targetArr = getFeaturesAndTargetsByWindowAndHop(finalChordDict, windowSize, hopSize)
#featureArr, targetArr = getFeaturesAndTargetsOfFlatSong(finalChordDict, featureLength)
print(featureArr.shape)
print(targetArr.shape)

Chords are shorter than number of targets so skipping the song: TwistAndShout
Chords are shorter than number of targets so skipping the song: ThatsAlrightMama
(1596, 16)
(1596, 1)
Slicing the song chords is done!
(1596, 16)
(1596, 1)


## Training Support Vector Machine Regression model

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn import svm
import sklearn
import pandas as pd

Here we first flatten the targets array and later we split the data by using %30 approach as test and training data.

In [24]:
targets = pd.Series(targetArr.flatten())

In [25]:
featuresTrain, featuresTest, targetTrain, targetTest = train_test_split(featureArr, targets, test_size=0.3, random_state=4)


Here we create a function which trains the SVR and returns the regression model. Later we will train many models and compare the performance by using this function.

We also have a timer in the training function which logs the training time.

In [26]:
def trainAndGetSVRModel(valueC, valueEpsilon, kernel, featuresTrain, targetTrain):
    startTime = time.time()
    regr = svm.SVR(C=valueC, epsilon=valueEpsilon, kernel=kernel)
    regr.fit(featuresTrain, targetTrain)
    endTime = time.time()
    timeSpent = endTime - startTime
    print(f"Time spent for the training is: {timeSpent:.4f} seconds, for values: " + str(valueC) + " epsilon: " + str(valueEpsilon) + " kernel " + kernel)
    return regr

**getErrorValues** function returns a row of different type of error values for the predicted values. This method used in order to print a table of different results.

In [27]:
def getErrorValues(targetTest, targetPredicted, paramName, paramValue):
    return { paramName: paramValue,
            'Mean Squared Err': '%.4f'% sklearn.metrics.mean_squared_error(targetTest, targetPredicted),
            'Mean Abs Err':  '%.4f'% sklearn.metrics.mean_absolute_error(targetTest, targetPredicted),
            'Max Err':  '%.4f'% sklearn.metrics.max_error(targetTest, targetPredicted),
            'Median Abs Err':  '%.4f'% sklearn.metrics.median_absolute_error(targetTest, targetPredicted),
            'R2 score':  '%.4f'% sklearn.metrics.r2_score(targetTest, targetPredicted),
            'Variance score':  '%.4f'% sklearn.metrics.explained_variance_score(targetTest, targetPredicted)}

In this cell we define a styling for our tables so that we can see the difference between errors better and compare better.

In [28]:
def highlight(value):
        return 'border: 1px solid black; padding: 5px'
    
def getDfWithStyle(df):
    return df.style.applymap(highlight)

#### Now we train our models with different C values from 0.05 to 50.

In [29]:
targetPredicted1 = trainAndGetSVRModel(0.05, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredicted2 = trainAndGetSVRModel(0.10, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredicted3 = trainAndGetSVRModel(1.0, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredicted4 = trainAndGetSVRModel(10.0, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredicted5 = trainAndGetSVRModel(50.0, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)

Time spent for the training is: 0.0807 seconds, for values: 0.05 epsilon: 0.001 kernel rbf
Time spent for the training is: 0.0783 seconds, for values: 0.1 epsilon: 0.001 kernel rbf
Time spent for the training is: 0.0835 seconds, for values: 1.0 epsilon: 0.001 kernel rbf
Time spent for the training is: 0.1277 seconds, for values: 10.0 epsilon: 0.001 kernel rbf
Time spent for the training is: 0.3128 seconds, for values: 50.0 epsilon: 0.001 kernel rbf


In [30]:
dataC = getErrorValues(targetTest, targetPredicted1, 'C Value', '0.05')
resultCDf = pd.DataFrame(dataC, index=[0])
resultCDf.loc[len(resultCDf)] = getErrorValues(targetTest, targetPredicted2, "C Value", '0.10')
resultCDf.loc[len(resultCDf)] = getErrorValues(targetTest, targetPredicted3, "C Value", '1.00')
resultCDf.loc[len(resultCDf)] = getErrorValues(targetTest, targetPredicted4, "C Value", '10.00')
resultCDf.loc[len(resultCDf)] = getErrorValues(targetTest, targetPredicted5, "C Value", '50.00')

display(getDfWithStyle(resultCDf))

,C Value,Mean Squared Err,Mean Abs Err,Max Err,Median Abs Err,R2 score,Variance score
0,0.05,16.2660,3.3697,7.0794,3.0793,0.0170,0.0253
1,0.10,15.8620,3.3386,7.1592,3.1594,0.0414,0.0478
2,1.00,14.3619,3.1093,9.1094,2.9239,0.1321,0.1334
3,10.00,14.8128,2.9338,12.0922,2.5114,0.1048,0.1052
4,50.00,17.6650,3.1051,13.5373,2.4461,-0.0675,-0.0662


#### Now we train the models with different kernel functions

In [31]:
targetPredictedByRbf = trainAndGetSVRModel(1.0, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedByLinear = trainAndGetSVRModel(1.0, 0.001, 'linear', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedByPoly = trainAndGetSVRModel(1.0, 0.001, 'poly', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedBySigmoid = trainAndGetSVRModel(1.0, 0.001, 'sigmoid', featuresTrain, targetTrain).predict(featuresTest)


Time spent for the training is: 0.0830 seconds, for values: 1.0 epsilon: 0.001 kernel rbf
Time spent for the training is: 0.4960 seconds, for values: 1.0 epsilon: 0.001 kernel linear
Time spent for the training is: 0.1971 seconds, for values: 1.0 epsilon: 0.001 kernel poly
Time spent for the training is: 0.1000 seconds, for values: 1.0 epsilon: 0.001 kernel sigmoid


In [32]:
dataKernel = getErrorValues(targetTest, targetPredictedByRbf, 'Kernel Value', 'rbf')
resultKernelDf = pd.DataFrame(dataKernel, index=[0])
resultKernelDf.loc[len(resultKernelDf)] = getErrorValues(targetTest, targetPredictedByLinear, 'Kernel Value','linear')
resultKernelDf.loc[len(resultKernelDf)] = getErrorValues(targetTest, targetPredictedByPoly, 'Kernel Value', 'poly')
resultKernelDf.loc[len(resultKernelDf)] = getErrorValues(targetTest, targetPredictedBySigmoid, 'Kernel Value', 'sigmoid')

display(getDfWithStyle(resultKernelDf))


,Kernel Value,Mean Squared Err,Mean Abs Err,Max Err,Median Abs Err,R2 score,Variance score
0,rbf,14.3619,3.1093,9.1094,2.9239,0.1321,0.1334
1,linear,14.8588,3.1918,9.1935,3.2074,0.1021,0.1036
2,poly,15.4463,3.1165,11.3957,2.7576,0.0666,0.0686
3,sigmoid,254.0218,11.5175,63.0430,8.4900,-14.3509,-13.7441


#### Now we train the models with different epsilon values from 0.001 to 0.1

In [33]:
targetPredictedByEps1 = trainAndGetSVRModel(1.0, 0.001, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedByEps2 = trainAndGetSVRModel(1.0, 0.005, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedByEps3 = trainAndGetSVRModel(1.0, 0.010, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedByEps4 = trainAndGetSVRModel(1.0, 0.050, 'rbf', featuresTrain, targetTrain).predict(featuresTest)
targetPredictedByEps5 = trainAndGetSVRModel(1.0, 0.100, 'rbf', featuresTrain, targetTrain).predict(featuresTest)


Time spent for the training is: 0.0831 seconds, for values: 1.0 epsilon: 0.001 kernel rbf
Time spent for the training is: 0.1142 seconds, for values: 1.0 epsilon: 0.005 kernel rbf
Time spent for the training is: 0.0820 seconds, for values: 1.0 epsilon: 0.01 kernel rbf
Time spent for the training is: 0.0788 seconds, for values: 1.0 epsilon: 0.05 kernel rbf
Time spent for the training is: 0.0773 seconds, for values: 1.0 epsilon: 0.1 kernel rbf


In [34]:
dataEpsion = getErrorValues(targetTest, targetPredictedByEps1, "Epsilon Value", '0.001')
resultEpsilonDf = pd.DataFrame(dataEpsion, index=[0])
resultEpsilonDf.loc[len(resultEpsilonDf)] = getErrorValues(targetTest, targetPredictedByEps2, "Epsilon Value", '0.005')
resultEpsilonDf.loc[len(resultEpsilonDf)] = getErrorValues(targetTest, targetPredictedByEps3, "Epsilon Value", '0.010')
resultEpsilonDf.loc[len(resultEpsilonDf)] = getErrorValues(targetTest, targetPredictedByEps4, "Epsilon Value", '0.050')
resultEpsilonDf.loc[len(resultEpsilonDf)] = getErrorValues(targetTest, targetPredictedByEps5, "Epsilon Value", '0.100')

display(getDfWithStyle(resultEpsilonDf))


,Epsilon Value,Mean Squared Err,Mean Abs Err,Max Err,Median Abs Err,R2 score,Variance score
0,0.001,14.3619,3.1093,9.1094,2.9239,0.1321,0.1334
1,0.005,14.3616,3.1093,9.1078,2.9212,0.1321,0.1335
2,0.010,14.3613,3.1093,9.1060,2.9205,0.1321,0.1335
3,0.050,14.3608,3.1103,9.0837,2.9396,0.1322,0.1335
4,0.100,14.3643,3.1116,9.0454,2.9217,0.1319,0.1332


## Train and Predict next n chords for given song

In this chapter, we will use the best performed values in the previous trainings and train a model with the feature and target arrays we extracted from The Beatles.

The goal is getting a song and composing a new chorus for the song by using this model.

First we need a function to fit the chords played before chorus in the song to our feature array rows. The function **prepareSongForPrediction** gets the chords played before the chorus, whole chords of the song and the feature length. 

If there is not enough chords played before the chorus, we add chords using the chords in the song starting by the end of the song. 
If the chords played before chorus is longer than our feature length, then we throw the chords away from the beginning.

In [35]:
def prepareSongForPrediction(beginningChords, fullChords, featureSize):
    finalArr = np.zeros((1, featureSize), dtype=int)
    lengthOfBeginning = len(beginningChords)
    lengthOfFullSong = len(fullChords)
    if lengthOfBeginning < featureSize:
        chordArray = np.zeros(featureSize, dtype=int)
        print('Beginning chords are shorter than number of features, repeating the chords')
        chordArray[featureSize-lengthOfBeginning:featureSize] = beginningChords
        if featureSize-lengthOfBeginning <= lengthOfFullSong:
            lengthToAdd = featureSize-lengthOfBeginning
            chordArray[0:lengthToAdd] = fullChords[lengthOfFullSong-lengthToAdd:lengthOfFullSong]
            chordArray[lengthToAdd:featureSize] = beginningChords
        else:
            quotient, remainder = divmod(featureSize-lengthOfBeginning, lengthOfFullSong) 
            print(quotient)
            print(remainder)
            chordArray[0:remainder] = chordArray[lengthOfFullSong-remainder:lengthOfFullSong]
            for step in range(quotient):
                start = remainder + step*lengthOfFullSong
                end = remainder + (step+1)*lengthOfFullSong
                chordArray[start:end] = fullChords
        finalArr[0:featureSize] = chordArray
    elif lengthOfBeginning > featureSize:
        finalArr[0:featureSize] = beginningChords[lengthOfBeginning-featureSize: lengthOfBeginning]
    else:
        finalArr[0:featureSize] = beginningChords
    print('Chords are ready for prediction.')
    return finalArr

#### Now we train the SVMR with C value of 1.0, epsilon of 0.001 and kernel of rbf

In [36]:
regression = trainAndGetSVRModel(1.0, 0.001, 'rbf', featuresTrain, targetTrain)

Time spent for the training is: 0.0873 seconds, for values: 1.0 epsilon: 0.001 kernel rbf


Here we decide which song we like to compose the chorus for. We can change the song name by changing the **songFileName**.

Later we create a Song object with the ChordPro file and we extract the full chords, the chords played before the chorus and the original chorus of the song.

In order to use the song for prediction, we need to map the chord names to integer again by using the same functions before.

In [37]:
#songFileName = 'AllYouNeedIsLove.chopro'
#songFileName = 'StrawberryFieldsForever.2.chopro'
songFileName = 'ObLaDiObLaDa.chopro'

songToGenerate = Song(pathOfChordTxtFiles + songFileName)
fullChords = songToGenerate.getChords()
chordsBeforeChorus = songToGenerate.getChordsBeforeChorus()

chordsBeforeChorusInt = mapChordNamesToIndexes(chordsBeforeChorus)
fullChordsInt = mapChordNamesToIndexes(fullChords)

featureOfSong = prepareSongForPrediction(chordsBeforeChorusInt, fullChordsInt, windowSize)

./data/chordpro/ObLaDiObLaDa.chopro
Beginning chords are shorter than number of features, repeating the chords
Chords are ready for prediction.


In this step, our feature array fronm the chosen song is ready to be used by the regression. 

We use **generatePredictedChords** function in order to call the predict function and add each chord predicted to the end of the chords played before the chorus. Every time we add the prediction to the end and remove a chord from the beginning. So that every prediction will be done on the chords together with predicted ones.

In [38]:
def generatePredictedChords(regr, beginningChords, chordsLengthToPredict):
    featureArrLength = len(beginningChords[0:])
    predictedChords = np.zeros(chordsLengthToPredict, dtype=int)
    for i in range(chordsLengthToPredict):
        nextValue = int(regr.predict(beginningChords))
        predictedChords[i] = nextValue
        beginningChords = np.roll(beginningChords, -1, axis=1)
        beginningChords[:, -1] = nextValue
        print(beginningChords)
    print('Done, chords predicted: ')
    print(predictedChords)
    return predictedChords

Now we run the functions and print both the original chorus and composed chorus by regression.

In [39]:
chorus = songToGenerate.getChorus()
chorusLengthToPredict = len(chorus)

composedChords = generatePredictedChords(regression, featureOfSong, chorusLengthToPredict)
chordNamesOfComposition = mapIndexesToChordNames(composedChords)

print('Original chorus: ', chorus)
print('Composed chorus: ', chordNamesOfComposition)


[[12 12  2  8 12  6 12 12  6 12 12  4 12  6 12  6]]
[[12  2  8 12  6 12 12  6 12 12  4 12  6 12  6  6]]
[[ 2  8 12  6 12 12  6 12 12  4 12  6 12  6  6  7]]
[[ 8 12  6 12 12  6 12 12  4 12  6 12  6  6  7  8]]
[[12  6 12 12  6 12 12  4 12  6 12  6  6  7  8  6]]
[[ 6 12 12  6 12 12  4 12  6 12  6  6  7  8  6  7]]
Done, chords predicted: 
[6 6 7 8 6 7]
Original chorus:  ['G', 'Bm', 'Em', 'G', 'D7', 'G']
Composed chorus:  ['D', 'D', 'D#', 'E', 'D', 'D#']


### Lets listen the midi file of composed chords

Lets convert the composition to a listenable format. In this case we will create a Acoustic Grand Piano which plays the chords where each chord is played for 500 milliseconds. 

The output will be saved as a midi file with the name of the song together with '-composed-chorus.mid' and it can be found and listened under the composed_midi folder.

In [40]:
from pychord import Chord
import pretty_midi


def createMidi(chordNamesOfComposition, songFileName):
    chords = [Chord(c) for c in chordNamesOfComposition]
    midi = pretty_midi.PrettyMIDI()
    pianoProgram = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=pianoProgram)
    length = 1
    for n, chord in enumerate(chords):
        for noteName in chord.components_with_pitch(root_pitch=4):
            noteNumber = pretty_midi.note_name_to_number(noteName)
            note = pretty_midi.Note(velocity=100, pitch=noteNumber, start=n * length, end=(n + 1) * length)
            piano.notes.append(note)
    midi.instruments.append(piano)
    midi.write('./composed_midi/' + songFileName+ '-composed-chorus.mid')


createMidi(chordNamesOfComposition, songFileName)

## Create an Http POST API

In order to examine the SVMR for real time algorithmic composition purposes, I have created an HTTP POST API.

In order to use the API, providing a flat array of chord names and the length of the new composed chords are the only values needed.

The API uses the chord array for training a new model by using window length of 16 and hop size of 2 and runs predictions for the same array.

The composed chords are returned as the response in array of chord names. In live coding or other algorithmic composition performances, this API can be called iteratively many times as it is responsing very quickly.

Also for us to listen, I have also created a midi file of piano instrument playing the chords in the api. So after the api is run, the idi file named composedArray can be listened.

In [41]:
def trainAndGenerateChords(chordArrayToTrain, numberOfResultsToPredict):
    windowSize = 16
    hopSize = 2
    
    dictOfSong = {"songName": chordArrayToTrain}
    featureArr, targetArr = getFeaturesAndTargetsByWindowAndHop(dictOfSong, windowSize, hopSize)
    targets = pd.Series(targetArr.flatten())
    featuresTrain, featuresTest, targetTrain, targetTest = train_test_split(featureArr, targets, test_size=0.3, random_state=4)
    
    regression = trainAndGetSVRModel(1.00, 0.001, 'rbf', featuresTrain, targetTrain)
    
    arrayForPrediction = prepareSongForPrediction(chordArrayToTrain, chordArrayToTrain, windowSize)
    composedChords = generatePredictedChords(regression, arrayForPrediction, numberOfResultsToPredict)

    return mapIndexesToChordNames(composedChords)
    

In [42]:
def trainAndCheckErrorValues(chordArrayToTrain, numberOfResultsToPredict):
    windowSize = 16
    hopSize = 2
    
    dictOfSong = {"songName": chordArrayToTrain}
    featureArr, targetArr = getFeaturesAndTargetsByWindowAndHop(dictOfSong, windowSize, hopSize)
    targets = pd.Series(targetArr.flatten())
    featuresTrain, featuresTest, targetTrain, targetTest = train_test_split(featureArr, targets, test_size=0.3, random_state=4)
    
    regression = trainAndGetSVRModel(1.00, 0.001, 'rbf', featuresTrain, targetTrain)
    targetsPredicted = regression.predict(featuresTest)
    
    resultApiDf = pd.DataFrame(getErrorValues(targetTest, targetsPredicted, 'Api composed', ''), index=[0])
    display(getDfWithStyle(resultApiDf))
    
    arrayForPrediction = prepareSongForPrediction(chordArrayToTrain, chordArrayToTrain, windowSize)
    composedChords = generatePredictedChords(regression, arrayForPrediction, numberOfResultsToPredict)

    return mapIndexesToChordNames(composedChords)

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/trainAndGenerate', methods=['POST'])
def trainSVRAndGenerateComposition():
    print('Api trainAndGenerate is called!')
    startTime = time.time()
    inputArrayOfChords = []
    numberOfResultsToPredict = 0
    try:
        data = request.json
        inputArrayOfChords = data['chordArayToTrain']
        numberOfResultsToPredict = data['numberOfResultsToPredict']
    except:
        print("An exception occurred")
        raise ValueError("Chords should be provided in a string array of chordArayToTrain")
    
    inputArrayOfChords = mapChordNamesToIndexes(inputArrayOfChords)
    #chordNamesOfComposition = trainAndGenerateChords(inputArrayOfChords, numberOfResultsToPredict)
    chordNamesOfComposition = trainAndCheckErrorValues(inputArrayOfChords, numberOfResultsToPredict)
    
    endTime = time.time()
    timeSpent = endTime - startTime
    print(f"Time spent for the  training and prediction is: {timeSpent:.4f} seconds")
    createMidi(chordNamesOfComposition, 'api composition')
    return jsonify({'composedArray': chordNamesOfComposition})

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Api trainAndGenerate is called!
(25, 16)
(25, 1)
Slicing the song chords is done!
Time spent for the training is: 0.0010 seconds, for values: 1.0 epsilon: 0.001 kernel rbf


,Api composed,Mean Squared Err,Mean Abs Err,Max Err,Median Abs Err,R2 score,Variance score
0,,18.7589,3.8559,6.3381,4.7689,0.0621,0.0696


127.0.0.1 - - [02/May/2023 10:35:20] "POST /trainAndGenerate HTTP/1.1" 200 -


Chords are ready for prediction.
[[ 6  4 12  0  6 12  0  6 12  2  8 12  4  6  4  6]]
[[ 4 12  0  6 12  0  6 12  2  8 12  4  6  4  6  6]]
[[12  0  6 12  0  6 12  2  8 12  4  6  4  6  6  6]]
[[ 0  6 12  0  6 12  2  8 12  4  6  4  6  6  6  8]]
[[ 6 12  0  6 12  2  8 12  4  6  4  6  6  6  8  6]]
[[12  0  6 12  2  8 12  4  6  4  6  6  6  8  6  6]]
[[ 0  6 12  2  8 12  4  6  4  6  6  6  8  6  6  7]]
[[ 6 12  2  8 12  4  6  4  6  6  6  8  6  6  7  6]]
Done, chords predicted: 
[6 6 6 8 6 6 7 6]
Time spent for the  training and prediction is: 0.0173 seconds
